<a href="https://colab.research.google.com/github/ptmhoang97/fiftyone_coco/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fiftyone

     |████████████████████████████████| 2.4 MB 9.3 MB/s 
     |████████████████████████████████| 29.2 MB 24 kB/s 
     |████████████████████████████████| 224 kB 45.0 MB/s 
     |████████████████████████████████| 106 kB 39.6 MB/s 
     |████████████████████████████████| 131 kB 44.8 MB/s 
     |████████████████████████████████| 557 kB 20.5 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 36.7 MB 64 kB/s 
     |████████████████████████████████| 569 kB 31.3 MB/s 
     |████████████████████████████████| 79.9 MB 49 kB/s 
     |████████████████████████████████| 13.2 MB 146 kB/s 
     |████████████████████████████████| 526 kB 43.7 MB/s 
     |████████████████████████████████| 76 kB 4.4 MB/s 
     |████████████████████████████████| 58 kB 6.0 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 77 kB 4.8 MB/s 
     |████████████████████████

In [2]:
import fiftyone

In [3]:
!fiftyone --version

fiftyone v0.11.2, Voxel51, Inc.


In [4]:
# The version that you wish to downgrade to
!fiftyone migrate --all -v 0.11.1

In [5]:
!fiftyone migrate --info

FiftyOne package version: 0.11.2
Database version: 0.11.2


In [6]:
!pip install fiftyone==0.11.1

     |████████████████████████████████| 2.4 MB 9.9 MB/s 
  Attempting uninstall: fiftyone
    Found existing installation: fiftyone 0.11.2
    Uninstalling fiftyone-0.11.2:
      Successfully uninstalled fiftyone-0.11.2


In [1]:
import fiftyone.zoo as foz

In [2]:
import os
print(os.name)

posix


In [3]:
COCO_DATASET_YEAR = '2017'
COCO_DATASET_TYPE = 'train'

In [27]:
paths = {
    'COCO_DATASET_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR)),
    'RESULT_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR), '_result'),
    'LABEL_TFRECORD_PATH': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR), '_label_tfrecord'),
    'SCRIPT_GEN_TFRECORD': os.path.join('fiftyone', 'coco-{}'.format(COCO_DATASET_YEAR), '_script_gen_tfrecord'),
 }

In [8]:
dataset = foz.load_zoo_dataset(name = 'coco-{}'.format(COCO_DATASET_YEAR),\
                               classes = "car",\
                               split = COCO_DATASET_TYPE,\
                               dataset_dir = paths['COCO_DATASET_PATH'])

 100% |██████|    1.9Gb/1.9Gb [6.2s elapsed, 0s remaining, 338.1Mb/s]       
Extracting annotations to 'fiftyone/coco-2017/raw/instances_train2017.json'
 100% |█████████████| 12251/12251 [43.5m elapsed, 0s remaining, 4.8 samples/s]      
Writing annotations for 12251 downloaded samples to 'fiftyone/coco-2017/train/labels.json'
Dataset info written to 'fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |█████████████| 12251/12251 [2.5m elapsed, 0s remaining, 84.3 samples/s]      
Dataset 'coco-2017-train' created


In [16]:
!mkdir {paths['RESULT_PATH']}

In [17]:
from pycocotools.coco import COCO
import os
import shutil
from tqdm import tqdm
# import skimage.io as io
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw

In [18]:
resultDir=paths['RESULT_PATH']
srcImageDir = paths['COCO_DATASET_PATH'] + '/{}/{}'.format(COCO_DATASET_TYPE,'data')
srcAnnoDir = paths['COCO_DATASET_PATH'] + '/{}'.format('raw')
datasets_list=[]
if COCO_DATASET_TYPE == 'validation':
    temp = 'val' + COCO_DATASET_YEAR
    datasets_list = [temp]
elif COCO_DATASET_TYPE == 'train':
    temp = 'train' + COCO_DATASET_YEAR
    datasets_list = [temp]
classes_names = ['car']
print("resultDir: " + str(resultDir))
print("srcImageDir: " + str(srcImageDir))
print("srcAnnoDir: " + str(srcAnnoDir))

resultDir: fiftyone/coco-2017/_result
srcImageDir: fiftyone/coco-2017/train/data
srcAnnoDir: fiftyone/coco-2017/raw


In [20]:
headstr = """\
<annotation>
    <folder>VOC</folder>
    <filename>%s</filename>
    <source>
        <database>My Database</database>
        <annotation>COCO</annotation>
        <image>flickr</image>
        <flickrid>NULL</flickrid>
    </source>
    <owner>
        <flickrid>NULL</flickrid>
        <name>company</name>
    </owner>
    <size>
        <width>%d</width>
        <height>%d</height>
        <depth>%d</depth>
    </size>
    <segmented>0</segmented>
"""
objstr = """\
    <object>
        <name>%s</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>%d</xmin>
            <ymin>%d</ymin>
            <xmax>%d</xmax>
            <ymax>%d</ymax>
        </bndbox>
    </object>
"""
 
tailstr = '''\
</annotation>
'''
 
#if the dir is not exists,make it,else delete it
def id2name(coco):
    classes=dict()
    for cls in coco.dataset['categories']:
        classes[cls['id']]=cls['name']
    return classes
 
def write_xml(anno_path,head, objs, tail):
    f = open(anno_path, "w")
    f.write(head)
    for obj in objs:
        f.write(objstr%(obj[0],obj[1],obj[2],obj[3],obj[4]))
    f.write(tail)
 
def save_annotations_and_imgs(coco,dataset,filename,objs):
    #eg:COCO_train2014_000000196610.jpg-->COCO_train2014_000000196610.xml
    result_anno_path = '{}/{}{}'.format(resultDir,filename[:-3],'xml')
    #print(result_anno_path)
    src_img_path = '{}/{}'.format(srcImageDir,filename)
    #print(anno_path)
    #print(src_img_path)
    dst_imgpath = '{}/{}'.format(resultDir,filename)
    #print(dst_imgpath)
    img=cv2.imread(src_img_path)
    #if (img.shape[2] == 1):
    #    print(filename + " not a RGB image")
     #   return
    shutil.copy(src_img_path, dst_imgpath)
 
    head=headstr % (filename, img.shape[1], img.shape[0], img.shape[2])
    tail = tailstr
    write_xml(result_anno_path,head, objs, tail)
 
 
def showimg(coco,dataset,img,classes,cls_id,show=True):
    #global dataDir
    path = '{}/{}'.format(srcImageDir,img['file_name'])
    #print(path)
    I=Image.open(path)
    #By id, get comment information
    annIds = coco.getAnnIds(imgIds=img['id'], catIds=cls_id, iscrowd=None)
    # print(annIds)
    anns = coco.loadAnns(annIds)
    # print(anns)
    # coco.showAnns(anns)
    objs = []
    for ann in anns:
        class_name=classes[ann['category_id']]
        if class_name in classes_names:
            #print(class_name)
            if 'bbox' in ann:
                bbox=ann['bbox']
                xmin = int(bbox[0])
                ymin = int(bbox[1])
                xmax = int(bbox[2] + bbox[0])
                ymax = int(bbox[3] + bbox[1])
                obj = [class_name, xmin, ymin, xmax, ymax]
                objs.append(obj)
                draw = ImageDraw.Draw(I)
                draw.rectangle([xmin, ymin, xmax, ymax])
    if show:
        plt.figure()
        plt.axis('off')
        plt.imshow(I)
        plt.show()
 
    return objs

for dataset in datasets_list:
    #./COCO/annotations/instances_train2014.json
    annFile='{}/instances_{}.json'.format(srcAnnoDir,dataset)
    #print(annFile)
    #COCO API for initializing annotated data
    coco = COCO(annFile)
 
    #show all classes in coco
    classes = id2name(coco)
    #print(classes)
    #[1, 2, 3, 4, 6, 8]
    classes_ids = coco.getCatIds(catNms=classes_names)
    #print(classes_ids)
    for cls in classes_names:
        #Get ID number of this class
        cls_id=coco.getCatIds(catNms=[cls])
        img_ids=coco.getImgIds(catIds=cls_id)
        #print(cls,len(img_ids))
        # imgIds=img_ids[0:10]
        for imgId in tqdm(img_ids):
            img = coco.loadImgs(imgId)[0]
            filename = img['file_name']
            # print(filename)
            objs=showimg(coco, dataset, img, classes,classes_ids,show=False)
            #print(objs)
            save_annotations_and_imgs(coco, dataset, filename, objs)

loading annotations into memory...
Done (t=26.70s)
creating index...


  0%|          | 7/12251 [00:00<03:19, 61.32it/s]

index created!


100%|██████████| 12251/12251 [03:19<00:00, 61.30it/s]


In [32]:
!zip -r '/content/fiftyone/coco-2017/_result.zip' '/content/fiftyone/coco-2017/_result'

Streaming output truncated to the last 5000 lines.
  adding: content/fiftyone/coco-2017/_result/000000489838.xml (deflated 82%)
  adding: content/fiftyone/coco-2017/_result/000000262736.jpg (deflated 1%)
  adding: content/fiftyone/coco-2017/_result/000000479815.jpg (deflated 0%)
  adding: content/fiftyone/coco-2017/_result/000000366385.jpg (deflated 0%)
  adding: content/fiftyone/coco-2017/_result/000000398442.xml (deflated 75%)
  adding: content/fiftyone/coco-2017/_result/000000447347.jpg (deflated 1%)
  adding: content/fiftyone/coco-2017/_result/000000079955.jpg (deflated 0%)
  adding: content/fiftyone/coco-2017/_result/000000399148.jpg (deflated 3%)
  adding: content/fiftyone/coco-2017/_result/000000016935.xml (deflated 60%)
  adding: content/fiftyone/coco-2017/_result/000000207072.xml (deflated 69%)
  adding: content/fiftyone/coco-2017/_result/000000533889.xml (deflated 60%)
  adding: content/fiftyone/coco-2017/_result/000000195079.xml (deflated 60%)
  adding: content/fiftyone/coco

In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [35]:
!cp /content/fiftyone/coco-2017/_result.zip /content/gdrive/MyDrive